In [126]:
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', 500)

## Consolidation Proof of Concept ##

In [127]:
#incs = pd.read_csv('milestone/01_nibrs_rawdata/UM_MSU_Offs.csv',parse_dates=['INCIDENT_DATE'])

In [128]:

# cbb = pd.read_csv('milestone/03_notebooks/cbb_2010_2019.csv', parse_dates=['game_dt','end_inc_window'])

In [129]:
import glob,os
directory = r'../02_sport_rawdata/cfbd_games_json/'
source_type = r'directory' # directory, .csv, or .xlsx
df = pd.concat(map(lambda file: pd.read_json(file), glob.glob(os.path.join('', directory + '*.json'))))

df_lst = []


def GetRankings(r):
    ''' This function takes a row of data and determines which poll to extract rankings from.  Note that this is not scaleable for large data due to looping of rows.
    '''
    

    tmp_df = pd.DataFrame(r['polls'])

    search_term = ""
    if r['season'] > 2013:
        if r['week'] < 10: 
            search_term = 'AP Top 25'
        else:
            search_term = 'Playoff Committee'
        
    elif r['season'] == 2013:
        if r['week'] < 9:
            search_term = 'AP Top 25'
        else:
            search_term = 'BCS'
    else:
        if r['week'] < 8:
            search_term = 'AP Top 25'
        else:
            search_term = 'BCS'
    
    if search_term == "":
        search_term = 'AP Top 25 i'
    rank_type = tmp_df[tmp_df['poll'].str.contains(search_term)==True]
    if rank_type.shape[0] == 0:
        search_term = 'AP Top 25'
        rank_type = tmp_df[tmp_df['poll'].str.contains('AP')==True]
    try:
        rankings = pd.json_normalize(rank_type.iloc[0]['ranks']).explode(['rank','school','conference'])
        # temp_ap = pd.json_normalize(r['polls']['poll']['ranks']).explode(['rank','school','conference'])
    except Exception:
        print(r['season'],r['week'],search_term)
        quit()
    rankings['season'] = r['season']
    rankings['season_type'] = r['seasonType']
    rankings['week'] = r['week']
    rankings['poll'] = search_term
    df_lst.append(rankings)
        

directory = r'../02_sport_rawdata/cfbd_poll_json/'
df_poll = pd.concat(map(lambda file: pd.read_json(file, orient='records'), glob.glob(os.path.join('', directory + '*.json'))))
df_poll.reset_index(inplace=True, drop=True)

# we don't want to return anything when we loop through the rows as there will be a list of dataframes stored we'll use in subsequent cells.
_ = df_poll.apply(lambda r: GetRankings(r),axis=1)



In [130]:
# Create the dataframe from the list of frames and validate the different polls that we're using and use some basic manipulation
cmb_ranks = pd.concat(df_lst)
cmb_ranks.poll.value_counts()
cmb_ranks['school'] = cmb_ranks['school'].str.lower()
#creating a duplicate column so that I can use that when merging for opponent ranks
cmb_ranks['opponent'] = cmb_ranks['school']
cmb_ranks

,rank,school,conference,firstPlaceVotes,points,season,season_type,week,poll,opponent
0,1,alabama,SEC,30,1503,2019,regular,8,AP Top 25,alabama
1,2,lsu,SEC,12,1449,2019,regular,8,AP Top 25,lsu
2,3,clemson,ACC,11,1427,2019,regular,8,AP Top 25,clemson
3,4,ohio state,Big Ten,9,1404,2019,regular,8,AP Top 25,ohio state
4,5,oklahoma,Big 12,0,1333,2019,regular,8,AP Top 25,oklahoma
...,...,...,...,...,...,...,...,...,...,...
20,2,ohio state,Big Ten,3,1414,2017,regular,1,AP Top 25,ohio state
21,11,michigan,Big Ten,0,881,2017,regular,1,AP Top 25,michigan
22,6,penn state,Big Ten,0,1196,2017,regular,1,AP Top 25,penn state
23,9,wisconsin,Big Ten,0,926,2017,regular,1,AP Top 25,wisconsin


In [131]:
df.columns

Index(['id', 'season', 'week', 'season_type', 'start_date', 'start_time_tbd',
       'neutral_site', 'conference_game', 'attendance', 'venue_id', 'venue',
       'home_id', 'home_team', 'home_conference', 'home_points',
       'home_line_scores', 'home_post_win_prob', 'home_pregame_elo',
       'home_postgame_elo', 'away_id', 'away_team', 'away_conference',
       'away_points', 'away_line_scores', 'away_post_win_prob',
       'away_pregame_elo', 'away_postgame_elo', 'excitement_index',
       'highlights', 'notes'],
      dtype='object')

In [132]:
# The football data is not specific to a school but rather a record of games. The Basketball data is generated by school and thus we have two 
# representations of the same game based on the school.  We need to translate the football data to a similar style.


def DetermineResult(h_a, home_points, away_points):
    if h_a == 'H':
        if home_points > away_points:
            return 'W'
        elif home_points < away_points:
            return 'L'
        else:
            return 'T'
    else:
        if home_points > away_points:
            return 'L'
        elif home_points < away_points:
            return 'W'
        else:
            return 'T'



um = df[(df['home_team']=='Michigan') | (df['away_team']=='Michigan')].copy()
um['school'] = 'michigan'
um['opponent'] = um.apply(lambda row: row['away_team'] if row['home_team']=='Michigan' else row['home_team'], axis=1)
um['h_a'] = um.apply(lambda row: 'H' if row['home_team']=='Michigan' else 'A', axis=1)


msu = df[(df['home_team']=='Michigan State') | (df['away_team']=='Michigan State')].copy()
msu['school'] = 'michigan state'
msu['opponent'] = msu.apply(lambda row: row['away_team'] if row['home_team']=='Michigan State' else row['home_team'], axis=1)
msu['h_a'] = msu.apply(lambda row: 'H' if row['home_team']=='Michigan State' else 'A', axis=1)

um_msu = pd.concat([um,msu])
um_msu['sport'] = 'football'

um_msu['result'] = um_msu.apply(lambda r: DetermineResult(r['h_a'],r['home_points'],r['away_points']),axis=1)
um_msu['team_points'] = um_msu.apply(lambda r: r['home_points'] if r['h_a'] == 'H' else r['away_points'], axis=1)
um_msu['opp_points'] = um_msu.apply(lambda r: r['home_points'] if r['h_a'] == 'A' else r['away_points'], axis=1)
um_msu['ot'] = um_msu.apply(lambda r: None if (len(r['home_line_scores']) -  4) == 0 else "OT" , axis=1)

um_msu = um_msu.merge(cmb_ranks[['rank','school','season','season_type','week']], how='left', on=['school','season','week','season_type'])
cmb_ranks['opponent_rank'] = cmb_ranks['rank']
um_msu.opponent = um_msu.opponent.str.lower()
um_msu = um_msu.merge(cmb_ranks[['opponent_rank','opponent','season','season_type','week']], how='left',on=['opponent','season','week','season_type'])
# um_msu['rank'] = um_msu['rank'].astype('int64')
# um_msu['opponent_rank'] = um_msu['opponent_rank'].astype('int64')


um_msu[um_msu['rank'].isna()==False]['rank'].unique()




# um_msu[['start_date','home_team','away_team','h_a','home_points','away_points','result','team_points','opp_points','ot']].sample(10)

array([20., 21., 19., 18.,  7., 11., 16., 14., 15., 13., 10., 25., 23.,
       22., 12., 17.,  5.,  4.,  3.,  2.,  6., 24.,  8.,  9.])

In [133]:
um_msu[['school','rank','opponent','opponent_rank','season','week','attendance']].sample(10)

,school,rank,opponent,opponent_rank,season,week,attendance
143,michigan state,10.0,penn state,NaN,2010,13,0.0
172,michigan state,2.0,purdue,NaN,2015,5,74418.0
239,michigan state,17.0,iowa,NaN,2011,11,0.0
99,michigan,22.0,san diego state,NaN,2011,4,0.0
68,michigan,NaN,nebraska,NaN,2013,11,112204.0
66,michigan,NaN,indiana,NaN,2013,8,109503.0
163,michigan state,NaN,iowa,6.0,2009,8,0.0
189,michigan state,NaN,illinois,NaN,2016,10,47144.0
217,michigan state,10.0,penn state,NaN,2014,14,99902.0
177,michigan state,13.0,maryland,NaN,2015,11,73406.0


In [134]:
um_msu[um_msu['start_date']=='2012-10-13T16:00:00.000Z'][['home_team','away_team','away_line_scores','ot']]

,home_team,away_team,away_line_scores,ot
249,Michigan State,Iowa,"[0, 3, 0, 10, 3, 3]",OT


In [135]:
um_msu['start_dt'] = pd.to_datetime(um_msu['start_date'])
um_msu['end_inc_window'] = um_msu['start_dt'] +  + pd.to_timedelta(11, unit='h')

In [136]:
um_msu = um_msu[['school','sport','start_dt','season_type','opponent','result','team_points','opp_points','ot','rank','opponent_rank','end_inc_window','attendance']]

In [137]:
um_msu.columns

Index(['school', 'sport', 'start_dt', 'season_type', 'opponent', 'result',
       'team_points', 'opp_points', 'ot', 'rank', 'opponent_rank',
       'end_inc_window', 'attendance'],
      dtype='object')

In [138]:
# Read in the college basketball data and do some quick manipulation to ensure columns match-up.

cbb = pd.read_csv('../04_finaldata/df_basketball_final.csv',parse_dates=['start_dt','end_inc_window'])
cbb = cbb[['school','sport','start_dt','type','opponent','result','team_points','opp_points','ot','rank','opponent_rank','end_inc_window']]
cbb = cbb.rename({'type': 'season_type'},axis=1)
cbb['opponent'] = cbb['opponent'].str.lower()
cbb['season_type'] = cbb['season_type'].apply(lambda x: "regular" if x=='REG' else x)
cbb.columns
# fbb = spark.createDataFrame(um_msu[['school','sport','game_dt','season_type','opponent','result','team_points','opp_points','ot','end_inc_window']])

Index(['school', 'sport', 'start_dt', 'season_type', 'opponent', 'result',
       'team_points', 'opp_points', 'ot', 'rank', 'opponent_rank',
       'end_inc_window'],
      dtype='object')

In [139]:
sports = pd.concat([um_msu,cbb])
sports['ot'] = sports.ot.replace([np.nan], [None])
sports = sports.reset_index(drop=True)
sports.index.names = ['game_id']
sports

,school,sport,start_dt,season_type,opponent,result,team_points,opp_points,ot,rank,opponent_rank,end_inc_window,attendance
game_id,,,,,,,,,,,,,
0,michigan,football,2010-09-04 19:30:00+00:00,regular,connecticut,W,30,10,None,NaN,NaN,2010-09-05 06:30:00+00:00,0.0
1,michigan,football,2010-09-11 19:30:00+00:00,regular,notre dame,W,28,24,None,NaN,NaN,2010-09-12 06:30:00+00:00,0.0
2,michigan,football,2010-09-18 16:00:00+00:00,regular,umass,W,42,37,None,20.0,NaN,2010-09-19 03:00:00+00:00,0.0
3,michigan,football,2010-09-25 16:00:00+00:00,regular,bowling green,W,65,21,None,21.0,NaN,2010-09-26 03:00:00+00:00,0.0
4,michigan,football,2010-10-02 19:30:00+00:00,regular,indiana,W,42,35,None,19.0,NaN,2010-10-03 06:30:00+00:00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
988,michigan-state,basketball,2019-03-21 14:45:00,NCAA,bradley,W,76,65,None,NaN,NaN,2019-03-22 00:45:00,NaN
989,michigan-state,basketball,2019-03-23 19:45:00,NCAA,minnesota,W,70,50,None,NaN,NaN,2019-03-24 05:45:00,NaN
990,michigan-state,basketball,2019-03-29 19:00:00,NCAA,louisiana state (12),W,80,63,None,NaN,12.0,2019-03-30 05:00:00,NaN


In [140]:
sports.columns

Index(['school', 'sport', 'start_dt', 'season_type', 'opponent', 'result',
       'team_points', 'opp_points', 'ot', 'rank', 'opponent_rank',
       'end_inc_window', 'attendance'],
      dtype='object')

In [141]:
sports = sports[['school','sport','start_dt','season_type','opponent','result','team_points','opp_points','ot','rank','opponent_rank','end_inc_window','attendance']]

In [142]:
sports.season_type.unique()

array(['regular', 'CTOURN', 'NCAA'], dtype=object)

In [143]:
sports.to_csv('../04_finaldata/sports.csv',header=True)

In [144]:
from pyspark.sql.types import TimestampType
from pyspark.sql.functions import col
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName('Merge') \
    .getOrCreate() 

sc = spark.sparkContext

In [145]:
sportsrdd = spark.read.csv('../04_finaldata/sports.csv',header=True,inferSchema=True)
sportsrdd = sportsrdd.withColumn('start_dt',col('start_dt').cast(TimestampType()))
sportsrdd = sportsrdd.withColumn('end_inc_window',col('end_inc_window').cast(TimestampType()))

In [146]:
sportsrdd.printSchema()

root
 |-- game_id: integer (nullable = true)
 |-- school: string (nullable = true)
 |-- sport: string (nullable = true)
 |-- start_dt: timestamp (nullable = true)
 |-- season_type: string (nullable = true)
 |-- opponent: string (nullable = true)
 |-- result: string (nullable = true)
 |-- team_points: integer (nullable = true)
 |-- opp_points: integer (nullable = true)
 |-- ot: string (nullable = true)
 |-- rank: string (nullable = true)
 |-- opponent_rank: double (nullable = true)
 |-- end_inc_window: timestamp (nullable = true)
 |-- attendance: double (nullable = true)



In [147]:
test = sportsrdd.sample(withReplacement=False,fraction=.1)

In [148]:
test.show()

+-------+--------------+--------+-------------------+-----------+-----------------+------+-----------+----------+----+----+-------------+-------------------+----------+
|game_id|        school|   sport|           start_dt|season_type|         opponent|result|team_points|opp_points|  ot|rank|opponent_rank|     end_inc_window|attendance|
+-------+--------------+--------+-------------------+-----------+-----------------+------+-----------+----------+----+----+-------------+-------------------+----------+
|      2|      michigan|football|2010-09-18 12:00:00|    regular|            umass|     W|         42|        37|null|20.0|         null|2010-09-18 23:00:00|       0.0|
|      9|      michigan|football|2010-11-13 12:00:00|    regular|           purdue|     W|         27|        16|null|null|         null|2010-11-13 23:00:00|       0.0|
|     25|      michigan|football|2009-09-12 15:30:00|    regular|       notre dame|     W|         38|        34|null|null|         18.0|2009-09-13 02:30:0

In [149]:
incs = spark.read.csv('../01_nibrs_rawdata/UM_MSU_Offs.csv', header=True, inferSchema=True)

In [150]:
incs = incs.withColumn('INCIDENT_DATE', col('INCIDENT_DATE').cast(TimestampType()))

In [151]:
sportsrdd.createOrReplaceTempView("sports")
incs.createOrReplaceTempView("incs")

In [152]:
query = """

SELECT
    s.game_id,
    s.sport, 
    s.school, 
    s.start_dt, 
    s.end_inc_window,
    s.rank,
    s.season_type,
    s.opponent,
    s.opponent_rank,
    s.result,
    s.team_points,
    s.opp_points,
    s.ot,
    s.attendance, 
    i.INCIDENT_ID as incident_id,
    i.OFFENSE_ID as offense_id,
    i.OFFENSE_NAME as offense_name,
    i.INCIDENT_DATE as incident_date

FROM
    sports s,
    incs i

WHERE
    i.INCIDENT_DATE BETWEEN s.start_dt AND s.end_inc_window AND
    i.school = s.school
"""


result = spark.sql(query)

In [153]:
df = result.toPandas()

In [154]:
df.shape

(1091, 18)

In [155]:
df.head()

,game_id,sport,school,start_dt,end_inc_window,rank,season_type,opponent,opponent_rank,result,team_points,opp_points,ot,attendance,incident_id,offense_id,offense_name,incident_date
0,67,football,michigan,2013-11-02 15:30:00,2013-11-03 01:30:00,21.0,regular,michigan state,22.0,L,6,29,None,76306.0,69063561,75222618,All Other Larceny,2013-11-02 22:00:00
1,395,basketball,michigan,2013-02-27 18:30:00,2013-02-28 04:30:00,4,regular,penn state,NaN,L,78,84,None,NaN,69057227,75221696,False Pretenses/Swindle/Confidence Game,2013-02-28 02:00:00
2,405,basketball,michigan,2013-04-06 21:21:00,2013-04-07 07:21:00,None,NCAA,syracuse (16),16.0,W,61,56,None,NaN,69058959,75221868,Drug/Narcotic Violations,2013-04-06 23:00:00
3,66,football,michigan,2013-10-19 15:30:00,2013-10-20 02:30:00,None,regular,indiana,NaN,W,63,47,None,109503.0,69063517,75222826,Intimidation,2013-10-20 02:00:00
4,383,basketball,michigan,2013-01-09 19:00:00,2013-01-10 05:00:00,2,regular,nebraska,NaN,W,62,47,None,NaN,69056264,75213130,Destruction/Damage/Vandalism of Property,2013-01-09 21:00:00


In [156]:
# take a quick look to see home many unique games had some type of incident associated to them
len(df.game_id.unique())

423

Now, since we performed an SQL operation that returned results that matched our criteria, we need to join these results to the games dataset.  We'll create another table view and call it 'game_incs' and use a 
left join to the sports data.

In [158]:
result.createOrReplaceTempView("game_incs")

In [159]:
query2 = """

SELECT s.*, g.incident_id,g.offense_id, g.offense_name, g.incident_date

FROM
    sports s LEFT JOIN game_incs g
    ON s.game_id = g.game_id
    
"""

result2 = spark.sql(query2)


In [160]:
full = result2.toPandas().set_index('game_id')

In [161]:
len(full.index.unique()), full.shape

(993, (1661, 17))

In [162]:
full.sample(20)

,school,sport,start_dt,season_type,opponent,result,team_points,opp_points,ot,rank,opponent_rank,end_inc_window,attendance,incident_id,offense_id,offense_name,incident_date
game_id,,,,,,,,,,,,,,,,,
608,michigan,basketball,2019-01-19 12:00:00,regular,wisconsin,L,54,64,None,2,NaN,2019-01-19 22:00:00,NaN,NaN,NaN,None,NaT
398,michigan,basketball,2013-03-10 16:00:00,regular,indiana (2),L,71,72,None,7,2.0,2013-03-11 02:00:00,NaN,69058046.0,75222129.0,Theft From Building,2013-03-10 21:00:00
90,michigan,football,2018-10-13 19:30:00,regular,wisconsin,W,38,13,None,12.0,15.0,2018-10-14 06:30:00,0.0,101845979.0,125530072.0,Destruction/Damage/Vandalism of Property,2018-10-14 02:00:00
589,michigan,basketball,2018-03-31 18:09:00,NCAA,loyola (il),W,69,57,None,None,NaN,2018-04-01 04:09:00,NaN,NaN,NaN,None,NaT
74,michigan,football,2014-09-13 15:30:00,regular,miami (oh),W,34,10,None,None,NaN,2014-09-14 02:30:00,102824.0,74955029.0,81997578.0,Drug/Narcotic Violations,2014-09-13 22:00:00
38,michigan,football,2015-09-19 12:00:00,regular,unlv,W,28,7,None,None,NaN,2015-09-19 23:00:00,108683.0,80208095.0,86827925.0,Drug/Narcotic Violations,2015-09-19 14:00:00
791,michigan-state,basketball,2014-01-25 19:00:00,regular,michigan (21),L,75,80,None,3,21.0,2014-01-26 05:00:00,NaN,NaN,NaN,None,NaT
257,michigan state,football,2017-09-23 20:00:00,regular,notre dame,L,18,38,None,None,NaN,2017-09-24 07:00:00,74023.0,93936214.0,116481381.0,Destruction/Damage/Vandalism of Property,2017-09-23 21:00:00
275,michigan,basketball,2009-12-13 12:00:00,regular,detroit mercy,W,75,64,None,-,NaN,2009-12-13 22:00:00,NaN,NaN,NaN,None,NaT


In [163]:
# Quick calculation to determine the number of sporting events that have an actual incident associated to them
grp = full.groupby('start_dt')[['incident_id','offense_name']].count()
grp[grp['incident_id'] > 0].shape[0] / 993

0.4068479355488419

In [164]:
full.to_csv('../04_finaldata/full_data.csv')

In [165]:
# Let's do a quick validation to make sure that the data associated ok. Take the game start_date with the highest number of incidents and check what offenses were created. Validate that we have the same number.
grp.sort_values('incident_id', ascending=False)

,incident_id,offense_name
start_dt,,
2014-10-25 15:30:00,14,14
2017-10-21 15:30:00,12,12
2011-11-19 12:00:00,12,12
2016-10-29 12:00:00,12,12
2011-10-15 12:00:00,11,11
...,...,...
2013-12-04 21:00:00,0,0
2013-12-07 12:00:00,0,0
2013-12-14 16:00:00,0,0


In [166]:
# Seeing 10-25-2014 has the highest number of offenses.  Let's see what incidents/offenses were generated
testsql = "SELECT i.school, i.INCIDENT_ID,i.OFFENSE_NAME FROM incs i WHERE i.INCIDENT_DATE BETWEEN '2014-10-25 15:30:00' AND '2014-10-26 2:30:00'"
r = spark.sql(testsql)
r.show()

+--------------+-----------+--------------------+
|        school|INCIDENT_ID|        OFFENSE_NAME|
+--------------+-----------+--------------------+
|      michigan|   74955899|Drug/Narcotic Vio...|
|michigan state|   73419507| Theft From Building|
|michigan state|   73418616|Destruction/Damag...|
|michigan state|   73418627|Destruction/Damag...|
|      michigan|   74955896| Theft From Building|
|michigan state|   73418619|      Simple Assault|
|michigan state|   73418637|      Simple Assault|
|michigan state|   73418625|      Simple Assault|
|michigan state|   73418636|      Simple Assault|
|michigan state|   73418620|      Simple Assault|
|michigan state|   73418617|  Aggravated Assault|
|michigan state|   73418618|      Simple Assault|
|      michigan|   74955897| Theft From Building|
|michigan state|   73418623|      Simple Assault|
+--------------+-----------+--------------------+



Looks like 14 offenses and most attributed on MSU's campus (assault).  This was a game that MSU and U-M played and MSU won, btw.

In [57]:
tstsql = "SELECT i.school, i.INCIDENT_ID,i.OFFENSE_NAME FROM incs i WHERE i.INCIDENT_DATE BETWEEN '2011-01-30 18:00:00' AND '2011-01-31 04:00:00'"
tresult = spark.sql(tstsql)
tresult.show()

+------+-----------+------------+
|school|INCIDENT_ID|OFFENSE_NAME|
+------+-----------+------------+
+------+-----------+------------+

